In [ ]:
using DataFrames
using Compat
using DataFramesMeta
using PyPlot

In [ ]:
include("../fund.jl/src/fund.jl")
include("../fund.jl/src/marginaldamages.jl")

Allocate some Dicts that hold the output per model. Key is model name

In [ ]:
σ_squareds = @compat Dict{String,Matrix{Float64}}()
cpcs = @compat Dict{String,Matrix{Float64}}()
pops = @compat Dict{String,Matrix{Float64}}()
globpops = @compat Dict{String,Vector{Float64}}()
mds = @compat Dict{String,Matrix{Float64}}()

# FUND

In [ ]:
const priceInflatorFUND = 1.55

## Get $\sigma^2$ for FUND

In [ ]:
x1 = readcsv("data/fund_sigmasquared.csv")
# Fix column/row layout
x2 = x1'
# Original uses a 5 year timestep, this repeats each year entry five times to get to a yearly matrix
x3 = repeat(x2, inner=[5,1])
# Create a matrix for later years that are not covered in the input file, by simply repeating the last year
x4 = repmat(x3[end,:], 195, 1)
# Combine the matrix for the covered years with the matrix for the later years
x5 = vcat(x3, x4)
# Done
σ_squareds["FUND"] = x5;

## Run FUND

In [ ]:
param_fund = loadparameters("../fund.jl/data");
m_fund = getfund(params=param_fund);
run(m_fund)
pops["FUND"] = m_fund[:population, :populationin1][65:65+284,:];
globpops["FUND"] = vec(sum(m_fund[:population, :populationin1], 2))[65:65+284]
cpcs["FUND"] = (m_fund[:socioeconomic, :consumption] ./ m_fund[:population, :populationin1])[65:65+284,:] .* priceInflatorFUND
mds["FUND"] = getmarginaldamages(parameters=param_fund, emissionyear=2015)[65:65+284,:];

# Compute SCC

Preallocate some arrays

In [ ]:
c_ede = similar(globpops["FUND"])
I = similar(mds["FUND"]);

Main equation of the paper

In [ ]:
function equation11(cpc, pop, globpop, md, σsquared, σZero, γ, η, ρ, α, focus_region, I, c_ede)
    timesteps, regions = size(cpc)
    
    for t=1:timesteps, r=1:regions
        if σZero
            I[t, r] = 1.0 - exp(-0.5 * γ * 0.0)
        else
            I[t, r] = 1.0 - exp(-0.5 * γ * σsquared[t, r])
        end
    end
      
    for t=1:timesteps
        if γ == 1.
            # DOUBLE CHECK
            temp_x = 0.
            for r=1:regions
                temp_x += pop[t,r] * log(cpc[t,r] * (1.0 - I[t,r]))
            end
            c_ede[t] = exp( temp_x / globpop[t])
        else
            temp_x = 0.
            for r=1:regions
                temp_x += pop[t,r] * (cpc[t,r] * (1.0 - I[t,r]))^(1.0 - γ)
            end
            c_ede[t] =(temp_x/globpop[t])^(1.0 / (1.0 - γ))
        end
    end

    globc = 0.
    for r=1:regions
        globc += pop[1,r] * cpc[1,r]
    end
    
    cpc0x = focus_region == -1 ? globc / globpop[1] : cpc[1, focus_region]

    scc = 0.
    for t=1:timesteps, r=1:regions
        scc +=  md[t, r] *                      # P_rt * d_rt in the paper
                (1.0 + ρ)^(-(t-1)) *          # time discount factor
                (cpc0x / cpc[t, r])^γ *         # Normal equity weight
                (c_ede[t] / c_ede[1])^(γ - η) * # New thing
                (1.0 - I[t, r])^(-(γ + 1.0)) *  # New thing 2
                (1.0 - I[t, r])^(2 * α)         # This is Δ
    end
    return scc
end;

Modify this to run for different parameter combinations.

In [ ]:
σZeros = [true, false]
#γs = linspace(0., 1.5, 16)
γs = linspace(0., 2., 11)
ηs = linspace(0., 2., 11)
#ρs = [0.001, 0.015, 0.03]
ρs = linspace(0., 0.06, 13)
αs = [0., 1.]
focus_regions = vcat(-1, [1:16]);

Compute SCCs

In [ ]:
df = DataFrame([Float64, Float64, Float64, Float64, Int64, Bool, Float64, String], [:rho, :eta, :gamma, :alpha, :focusregion, :sigmazero, :scc, :model], 0)

for model=keys(cpcs)
    cpc = cpcs[model]
    pop = pops[model]
    globpop = globpops[model]
    md = mds[model]
    σSquared = σ_squareds[model]

    for σZero=σZeros, γ=γs, η=ηs, ρ=ρs, α=αs, focus_region=focus_regions
        scc = equation11(cpc, pop, globpop, md, σSquared, σZero, γ, η, ρ, α, focus_region, I, c_ede)
        push!(df, (ρ,η,γ,α,focus_region,σZero,scc,model))
    end
end

In [ ]:
df

# Plotting

## Figure 1

In [ ]:
df_temp = @where(df, (:rho .== 0.015) & (:focusregion .== -1) & (:alpha .== 1.) & (:gamma .<= 1.5) )

df_figure1_panelA_line1 = @where(df_temp, (:model .== "FUND") & (:sigmazero .== true) & (:eta .== :gamma) )
df_figure1_panelA_line2 = @where(df_temp, (:model .== "FUND") & (:sigmazero .== true) & (:eta .== 1.4) )

df_figure1_panelB_line1 = @where(df_temp, (:model .== "RICE") & (:sigmazero .== true) & (:eta .== :gamma) )
df_figure1_panelB_line2 = @where(df_temp, (:model .== "RICE") & (:sigmazero .== true) & (:eta .== 1.4) )

df_figure1_panelC_line1 = @where(df_temp, (:model .== "FUND") & (:sigmazero .== false) & (:eta .== :gamma) )
df_figure1_panelC_line2 = @where(df_temp, (:model .== "FUND") & (:sigmazero .== false) & (:eta .== 1.4) )

df_figure1_panelD_line1 = @where(df_temp, (:model .== "RICE") & (:sigmazero .== false) & (:eta .== :gamma) )
df_figure1_panelD_line2 = @where(df_temp, (:model .== "RICE") & (:sigmazero .== false) & (:eta .== 1.4) )

In [ ]:
f, axs = subplots(2,2, sharex= true, sharey=true)

axs[1,1][:plot](df_figure1_panelA_line1[:gamma], df_figure1_panelA_line1[:scc])
axs[1,1][:plot](df_figure1_panelA_line2[:gamma], df_figure1_panelA_line2[:scc])
axs[1,1][:set_title]("FUND")

axs[1,2][:plot](df_figure1_panelB_line1[:gamma], df_figure1_panelB_line1[:scc])
axs[1,2][:plot](df_figure1_panelB_line2[:gamma], df_figure1_panelB_line2[:scc])
axs[1,2][:set_title]("RICE")

axs[2,1][:plot](df_figure1_panelC_line1[:gamma], df_figure1_panelC_line1[:scc])
axs[2,1][:plot](df_figure1_panelC_line2[:gamma], df_figure1_panelC_line2[:scc])
axs[2,1][:set_xlabel](L"$\gamma$")


axs[2,2][:plot](df_figure1_panelD_line1[:gamma], df_figure1_panelD_line1[:scc])
axs[2,2][:plot](df_figure1_panelD_line2[:gamma], df_figure1_panelD_line2[:scc])
axs[2,2][:set_xlabel](L"$\gamma$")